In [1]:
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint
import re

In [2]:
# Сюда вводим искомую должность
inp_vacancy = 'Java junior'

# А сюда колличество страниц для сбора данных с сайта 
num_of_pages = 4

In [3]:
vacancy_list = []
page = 1

In [4]:
url = 'https://www.hh.ru'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36'}
page_button = f'/search/vacancy?fromSearchLine=true&text={inp_vacancy}&from=suggest_post&search_field=name&page={page}'

In [5]:
%%time
for _ in range(num_of_pages):
    try:
        response = requests.get(url+page_button, headers=headers)
        dom = bs(response.text, 'html.parser')
        vacancies = dom.find_all('div', {'class': 'vacancy-serp-item'})
        for vacancy in vacancies:

            base = {}
            base['site'] = url
            base['title'] = vacancy.find('a', {'class': 'bloko-link', 'data-qa':"vacancy-serp__vacancy-title"}).text
            base['link'] = vacancy.find('a', {'class': 'bloko-link', 'data-qa':"vacancy-serp__vacancy-title"})['href']

            try:
                raw_salary = (vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'}).text)
                li = re.findall(r"\d+", raw_salary)

                if raw_salary[0] == 'о':
                    base['min_salary'] = int(li[0]+li[1])
                    base['max_salary'] = None
                    base['value'] = re.findall(r'\w+\.?$', raw_salary)[0]

                elif raw_salary[0] == 'д':
                    base['min_salary'] = None
                    base['max_salary'] = int(li[0]+li[1])
                    base['value'] = re.findall(r'\w+\.?$', raw_salary)[0]

                else:
                    base['min_salary'] = int(li[0]+li[1])
                    base['max_salary'] = int(li[2]+li[3])
                    base['value'] = re.findall(r'\w+\.?$', raw_salary)[0]

            except:
                base['min_salary'] = None
                base['max_salary'] = None
                base['value'] = None


            vacancy_list.append(base)
        
        page += 1
        
    except:
        break

Wall time: 5.49 s


In [6]:
pprint(vacancy_list)

[{'link': 'https://ufa.hh.ru/vacancy/49070669?from=vacancy_search_list&query=Java%20junior',
  'max_salary': None,
  'min_salary': None,
  'site': 'https://www.hh.ru',
  'title': 'Junior Java Developer (SBB)',
  'value': None},
 {'link': 'https://ufa.hh.ru/vacancy/49085666?from=vacancy_search_list&query=Java%20junior',
  'max_salary': None,
  'min_salary': None,
  'site': 'https://www.hh.ru',
  'title': 'Junior Java developer',
  'value': None},
 {'link': 'https://ufa.hh.ru/vacancy/42636573?from=vacancy_search_list&query=Java%20junior',
  'max_salary': 50000,
  'min_salary': 35000,
  'site': 'https://www.hh.ru',
  'title': 'Backend-разработчик Kotlin/Java (junior)',
  'value': 'руб.'},
 {'link': 'https://ufa.hh.ru/vacancy/48116107?from=vacancy_search_list&query=Java%20junior',
  'max_salary': None,
  'min_salary': None,
  'site': 'https://www.hh.ru',
  'title': 'Инженер-программист Java (Junior)',
  'value': None},
 {'link': 'https://ufa.hh.ru/vacancy/49102929?from=vacancy_search_list&

In [7]:
len(vacancy_list)

80